In [2]:
from openai import OpenAI
import os
from PyPDF2 import PdfReader
from pptx import Presentation
import ast

In [3]:
def pdf_to_string(file_path):
    pdf = PdfReader(file_path)

    text = ''

    for page in pdf.pages:
        text += page.extract_text()

    return text

In [4]:
company_details = pdf_to_string('data/company_details.pdf')

In [5]:
pdf_string = pdf_to_string('data/sample-rfp.pdf')

In [34]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


def proposal_generator(pdf_string, company_details):
    format = {
        "slide title": "<Title of the Presentation>",
        "Understanding of the Scope": [
            "<Point 1 for Slide 1>",
            "<Point 2 for Slide 1>",
            "<Point 3 for Slide 1>",
            "<Placeholder for additional points>",
        ],
        "Understanding of the Scope": [
            "<Point 1 for Slide 2>",
            "<Point 2 for Slide 2>",
            "<Point 3 for Slide 2>",
            "<Placeholder for additional points>",
        ],
        "Understanding of the Scope": [
            "<Point 1 for Slide 3>",
            "<Point 2 for Slide 3>",
            "<Point 3 for Slide 3>",
            "<Placeholder for additional points>",
        ],
        "Our Solution/Reference Architecture": [
            "<Point 1 for Slide 4>",
            "<Point 2 for Slide 4>",
            "<Placeholder for additional points>",
        ],
        "Our Solution/Reference Architecture": [
            "<Point 1 for Slide 5>",
            "<Point 2 for Slide 5>",
            "<Placeholder for additional points>",
        ],
        "Tech Stack Recommendation": [
            "<Point 1 for Slide 6>",
            "<Point 2 for Slide 6>",
            "<Placeholder for additional points>",
        ],
        "Tech Stack Recommendation": [
            "<Point 1 for Slide 7>",
            "<Point 2 for Slide 7>",
            "<Placeholder for additional points>",
        ],
        "Comparison/Rationale for all the suggested tool": [
            "<Point 1 for Slide 8>",
            "<Point 2 for Slide 8>",
            "<Point 3 for Slide 8>",
            "<Placeholder for additional points>",
        ],
        "Comparison/Rationale for all the suggested tool": [
            "<Point 1 for Slide 9>",
            "<Point 2 for Slide 9>",
            "<Point 3 for Slide 9>",
            "<Placeholder for additional points>",
        ],
        "Comparison/Rationale for all the suggested tool": [
            "<Point 1 for Slide 10>",
            "<Point 2 for Slide 10>",
            "<Point 3 for Slide 10>",
            "<Placeholder for additional points>",
        ],
    }

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are a helpful sales and business development assistant who creates narrative/content-driven presentations with good context which provides output strictly in json format.",
            },
            {
                "role": "user",
                "content": "RFP/Proposal : {pdf_string}\n\
                My company Details:{company_details}\n\
                Prompt : Making use of the provided RFP and company details,\n\
                create a proposal, a detailed narrative/content-driven presentation with the following content :\n\
                It should have a title slide according to the RFP.\n\
                1. Understanding of the Scope (2 slides)\n\
                2. Our Solution/Reference Architecture (2 slide)\n\
                3. Tech Stack Recommendation (1 slide)\n\
                5. Comparison/Rationale for all the suggested tool (2 slide).\n\
                Presentation should be structured in the following format with title and other slides : {format}\n\
                and it should be detailed with multiple slides for each title,\n\
                each slide should not have more than 3 points",
            },
        ],
    )
    return response.choices[0].message.content

In [35]:
slides_content = proposal_generator(pdf_string,company_details)

In [36]:
slides_content

'{\n  "Proposal": {\n    "Title": "Innovative Solutions Proposal for [Client Name] RFP",\n    "Slides": [\n      {\n        "Title": "Understanding of the Scope",\n        "Content": [\n          {\n            "Slide 1": [\n              "Detailed analysis of the requirements outlined in the RFP",\n              "Identification of key challenges and pain points to be addressed"\n            ]\n          },\n          {\n            "Slide 2": [\n              "Strategic approach to meet the project objectives",\n              "Proposed timeline and deliverables"\n            ]\n          }\n        ]\n      },\n      {\n        "Title": "Our Solution/Reference Architecture",\n        "Content": [\n          {\n            "Slide 1": [\n              "Overview of our proposed solution",\n              "High-level architecture diagram showcasing the system components"\n            ]\n          },\n          {\n            "Slide 2": [\n              "Case studies or examples of similar 

In [33]:
prs = Presentation()

presentation_content_dict = ast.literal_eval(slides_content)
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
title_placeholder = slide.shapes.title
title_placeholder.text = presentation_content_dict["slide title"]

for title, content in list(presentation_content_dict.items())[1:]:
    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)

    slide.shapes.title.text = title
    for line in content:
        p = slide.placeholders[1].text_frame.add_paragraph()
        p.text = line

# Save the presentation to a local file
prs.save('new_final-ai.pptx')

KeyError: 'slide title'

In [2]:
#images, logos (skills), no way to understand properly for AI, need to convert it to words
#company details fine tuning better?
#caching to save token usage (since the understanding doesnt vary), but the extra prompt user provides might need extra context?


#token usage is high at the moment

In [ ]:
#Understanding of the scope ()
# Our solution/ reference architecture (layers, features etc) without specific tech and things
# tech stack recommendation??
# comparison/rationale in line with the scope and also our company capabilities included in recommendation
# How we can help (Why mitz for this. Our similar case studies etc)